In [ ]:
### Alt method for keyword creation

# source: https://towardsdatascience.com/keyword-extraction-process-in-python-with-natural-language-processing-nlp-d769a9069d5c
import yake

def best_keywords (data, words_per_paper):
    a = data['citations'].quantile(q = .9)
    best = data[data['citations'] > a]
    best['keywords'] = ''

    extractor = yake.KeywordExtractor()
    language = 'en'
    max_ngram = 1
    deduplication_threshold = 0.1   # low value = duplication of keywords in ngrams not allowed
    num_keywords = words_per_paper


    for index, i_paper in best.iterrows(): # iterate over the dataframe 
        text = i_paper['abstract']
        custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram, dedupLim=deduplication_threshold, 
                                                    top=num_keywords, features=None)
        keywords = custom_kw_extractor.extract_keywords(text)

        k = []
        for i in keywords:
            k.append(i[0].lower())

        best.at[index, 'keywords'] = k

    a = best['keywords'].tolist()
    a = [x for l in a for x in l]
    
    return a


print(best_keywords(play, 3))




In [ ]:
"""
Remove outliers
NOTE: can't rerun this code without restarting the kernal
"""
#names: X_train, X_val, y_train, y_val
#print(list(X_train.columns))

# print("citations:", find_outliers_tukey(x = y_train['citations'], top = 93, bottom = 0))

# print("year:", find_outliers_tukey(X_train['year'], top = 74, bottom = 25))  # seems unnecessary
# print("references:", find_outliers_tukey(X_train['references'], top = 90, bottom = 10))  # seems unnecessary
# print("team_size:", find_outliers_tukey(X_train['team_size'], top = 99, bottom = 0))  # Meh
# print("topic_variety:", find_outliers_tukey(X_train['topic_variety'], top = 75, bottom = 10))  # not much diff btw top and normal
# print("age:", find_outliers_tukey(X_train['age'], top = 90, bottom = 10))  # Meh
# print("open_access:", find_outliers_tukey(X_train['open_access'], top = 100, bottom = 0))  # Not necessary: boolean
# print("has_keyword:", find_outliers_tukey(X_train['has_keyword'], top = 100, bottom = 0))  # Not necessary: boolean
# print("title_length:", find_outliers_tukey(X_train['title_length'], top = 90, bottom = 10))  # Meh
# print("field_variety:", find_outliers_tukey(X_train['field_variety'], top = 90, bottom = 10))  # seems unnecessary
# print("venue_freq:", find_outliers_tukey(X_train['venue_freq'], top = 90, bottom = 10))  # seems unnecessary


out_y = (find_outliers_tukey(x = y_train['citations'], top = 95, bottom = 0))[0]
#out_X = (find_outliers_tukey(x = X_train['team_size'], top = 99, bottom = 0))[0]
out_rows = out_y
#out_rows = out_y + out_X
out_rows = sorted(list(set(out_rows)))

print("X_train:")
print(X_train.shape)
X_train = X_train.drop(labels = out_rows)
print(X_train.shape)
print()
print("y_train:")
print(y_train.shape)
y_train = y_train.drop(labels = out_rows)
print(y_train.shape)




In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_z = scaler.fit_transform(X_train_small)
X_val_z  =scaler.transform(X_val_small)

polynomial_features = PolynomialFeatures(degree = 2)
x_train_poly = polynomial_features.fit_transform(X_train_z)
x_val_poly = polynomial_features.transform(X_val_z)

model = LinearRegression()
model.fit(x_train_poly, y_train)
y_poly_pred = model.predict(x_val_poly)

print(r2_score(y_val, y_poly_pred))   # -0.04350391168707901
print(mean_absolute_error(y_val, y_poly_pred))    # 32.65668266590838